In [2]:
def rnkOrdering(data_org, bad_col, wgt_col, score_col, n_bkts):
    if wgt_col is None:
        wgt_col = 'wt'
        data_org[wgt_col] = 1
    #data_org[score_col] = -1 * data_org[score_col]
    data = data_org.sort_values(by=[score_col]).reset_index()
    data["cum_wgt"] = data[wgt_col].cumsum()
    total = data[wgt_col].sum()
    bin_lvl = 0.1
    flag = 1
    for i in range(len(data)):
        if (data.loc[i, "cum_wgt"] >= total * bin_lvl) and (bin_lvl < 0.9):
            bin_lvl = bin_lvl + 0.1
            flag = flag + 1
            data.loc[i, "bucket"] = flag
        else:
            data.loc[i, "bucket"] = flag
    # data['bucket'] = pd.qcut(data[score_col], n_bkts)
    grouped = data.groupby('bucket', as_index=True)
    data["labels"] = data[bad_col] * data[wgt_col]
    agg1 = grouped[score_col].min()
    agg2 = grouped[score_col].max()
    agg3 = grouped[wgt_col].sum()
    agg4 = grouped["labels"].sum()
    agg5 = np.round(grouped[score_col].mean() * 100, 2)
    ret_tbl = pd.DataFrame({"min_score": agg1, "max_score": agg2, "total": agg3, "Bads": agg4, "Mean_Score": agg5})
    ret_tbl["Goods"] = ret_tbl["total"] - ret_tbl["Bads"]
    ret_tbl["Bad_Rate"] = np.round(ret_tbl["Bads"] / ret_tbl["total"], 4) * 100
    ret_tbl = ret_tbl.sort_index(axis=0, ascending=False)
    ret_tbl.index = range(n_bkts)

    ret_tbl["Cum_Bad"] = np.round((ret_tbl.Bads / ret_tbl.Bads.sum()).cumsum(), 4) * 100
    ret_tbl["Cum_Good"] = np.round((ret_tbl.Goods / ret_tbl.Goods.sum()).cumsum(), 4) * 100
    ret_tbl['KS'] = np.abs(ret_tbl["Cum_Bad"] - ret_tbl["Cum_Good"])

    gini = ret_tbl["Cum_Bad"][0] * ret_tbl["Cum_Good"][0] / (2 * 100 * 100)
    for i in range(1, n_bkts):
        gini += (ret_tbl["Cum_Bad"][i] + ret_tbl["Cum_Bad"][i - 1]) * (ret_tbl["Cum_Good"][i] -
                                                                       ret_tbl["Cum_Good"][i - 1]) / (2 * 100 * 100)
    gini = np.round(2. * gini - 1., 4) * 100

    ks = np.round(ret_tbl.KS.max(), 2)

    coltitles = ["min_score", "max_score", "Mean_Score", "total", "Goods", "Bads", "Bad_Rate", "Cum_Bad", "Cum_Good", "KS"]

    return ret_tbl[coltitles], {"Gini": gini, "KS": ks}